In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

In [ ]:
pip install pandas plotly dash


In [ ]:
df = pd.read_csv("netflix_titles.csv")
df

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...
...,...,...,...,...,...,...,...,...,...,...,...,...
8802,s8803,Movie,Zodiac,David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",United States,"November 20, 2019",2007,R,158 min,"Cult Movies, Dramas, Thrillers","A political cartoonist, a crime reporter and a..."
8803,s8804,TV Show,Zombie Dumb,NaN,NaN,NaN,"July 1, 2019",2018,TV-Y7,2 Seasons,"Kids' TV, Korean TV Shows, TV Comedies","While living alone in a spooky town, a young g..."
8804,s8805,Movie,Zombieland,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",United States,"November 1, 2019",2009,R,88 min,"Comedies, Horror Movies",Looking to survive in a world taken over by zo...
8805,s8806,Movie,Zoom,Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",United States,"January 11, 2020",2006,PG,88 min,"Children & Family Movies, Comedies","Dragged from civilian life, a former superhero..."


In [ ]:
for col in df.columns:
  print("Number of nulls in",col,":",df[col].isna().sum())

Number of nulls in show_id : 0
Number of nulls in type : 0
Number of nulls in title : 0
Number of nulls in director : 2634
Number of nulls in cast : 825
Number of nulls in country : 831
Number of nulls in date_added : 10
Number of nulls in release_year : 0
Number of nulls in rating : 4
Number of nulls in duration : 3
Number of nulls in listed_in : 0
Number of nulls in description : 0


In [ ]:
for col in df.columns:
    if df[col].isna().sum() > 0:
        mode_value = df[col].mode().iloc[0]
        df[col] = df[col].fillna(mode_value)

In [ ]:
app = dash.Dash(__name__)

# Create a function to generate visualizations based on selected genre
def generate_genre_visualization(selected_genre):
    genre_counts = df[df['listed_in'] == selected_genre]['type'].value_counts()
    genre_fig = go.Figure(data=[go.Bar(x=genre_counts.index, y=genre_counts.values)])
    genre_fig.update_layout(title=f'Distribution of Content Types in "{selected_genre}" Genre',
                            xaxis_title='Content Type', yaxis_title='Count')
    return genre_fig

# Create a function to generate a line chart for release years
def generate_release_year_chart():
    release_year_counts = df['release_year'].value_counts().sort_index()
    release_year_fig = go.Figure(data=[go.Scatter(x=release_year_counts.index, y=release_year_counts.values, mode='lines+markers')])
    release_year_fig.update_layout(title='Number of Movies/TV Shows Released Each Year',
                                   xaxis_title='Year', yaxis_title='Count')
    return release_year_fig

# Create a function to generate a pie chart for ratings
def generate_rating_pie_chart():
    rating_counts = df['rating'].value_counts()
    rating_fig = go.Figure(data=[go.Pie(labels=rating_counts.index, values=rating_counts.values)])
    rating_fig.update_layout(title='Distribution of Ratings')
    return rating_fig

# Define the layout of the dashboard
app.layout = html.Div([
    html.H1("Netflix Movies and TV Shows Dashboard"),

    # Dropdown menu to select the genre
    dcc.Dropdown(
        id='genre-dropdown',
        options=[{'label': genre, 'value': genre} for genre in df['listed_in'].unique()],
        value='Action',
        placeholder='Select a Genre',
    ),

    # Interactive bar chart for movie genres
    dcc.Graph(id='genre-bar-chart'),

    # Interactive line chart for release years
    dcc.Graph(id='release-year-line-chart', figure=generate_release_year_chart()),

    # Interactive pie chart for ratings
    dcc.Graph(id='rating-pie-chart', figure=generate_rating_pie_chart())
])

# Callback to update genre bar chart based on selected genre
@app.callback(
    Output('genre-bar-chart', 'figure'),
    [Input('genre-dropdown', 'value')]
)
def update_genre_chart(selected_genre):
    return generate_genre_visualization(selected_genre)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>